# 02 - Data from the Web

We start importing all the libraries that we are going to use.

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup # if you don't have this library, 
# install it. The homework instructions tell us to use it.


Generating the base of our futures requests

In [3]:
payloadform = {'ww_i_reportModel': '133685247'}
isa_baseurl = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS'
isa_formurl= isa_baseurl + ".filter"
r = requests.get(isa_formurl, params=payloadform)
s = r.url + " " + str(r.status_code)
s

'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247 200'

In [4]:
soup = BeautifulSoup(r.text, 'html.parser')
inputs = soup.select('form input') # extraemos una lista con todos los 
# inputs del form ,los 4 primeros inputs no nos valen
inputs[5]['name'] # nombre de la opcion
tags=inputs[6].select('option') # extraemos las posibles opcicones 
# de los inputs 
tags[1].contents # mostramos el contenido o texto dentro de una opcion
tags[1]['value'] # valor que meter en nuestra petición rest (de cada opcion)
inputs[5].select('option')[9].contents[0] # extraemos los nombres de los inputs

'Informatique'

With the data that we have just obtain, we will make the request petitions with a loops


In [52]:
payloadfix = {'ww_x_GPS': '-1', 'ww_i_reportModel': '133685247', 'ww_i_reportModelXsl': '133685270', 'ww_x_UNITE_ACAD': '','ww_x_PERIODE_ACAD': '', 'ww_x_PERIODE_PEDAGO': '','ww_x_HIVERETE': '' }
isa_dataurl = isa_baseurl + ".html"
# We have to make a loop here changing the payloadfix values. (only the next ones)
payloadfix['ww_x_UNITE_ACAD'] = inputs[5].select('option')[9]['value']
#payloadfix['ww_x_PERIODE_ACAD'] = inputs[6].select('option')[10]['value']#decrement [10]==2007
#payloadfix['ww_x_PERIODE_PEDAGO'] = inputs[7].select('option')[1]['value']#[7]==bach6
#payloadfix
#r = requests.get(isa_dataurl, params=payloadfix)
#r.url


In [ ]:
# Here we will need to parse the html using BeautifulSoup library.
# The data that we want comes in a html table, won't be really hard to parse it.
#Data = BeautifulSoup(r.text, 'html.parser') 


def extractTable(year,sem):
    payloadfix['ww_x_PERIODE_ACAD'] = inputs[6].select('option')[year]['value']
    payloadfix['ww_x_PERIODE_PEDAGO'] = inputs[7].select('option')[sem]['value']#[7]==bach6
    if(sem==1):
        payloadfix['ww_x_HIVERETE'] = inputs[8].select('option')[1]['value']
    else:
        payloadfix['ww_x_HIVERETE'] = inputs[8].select('option')[2]['value']
    
    r = requests.get(isa_dataurl, params=payloadfix)
    Data = BeautifulSoup(r.text, 'html.parser') 
    """Extract the table from HTML into a pandas Dataframe."""
    BStable = BeautifulSoup(r.text, 'html.parser').find('table')
    
    BSRows = BStable.findAll('tr')
    # BSrows[0] is just the title of the table, so we ignore it.
    headerBSRow = BSRows[1]
    headers = [th.string for th in headerBSRow]
    # We drop the last column, because it's always empty (originally it is used to make the HTML table look better).
    rows = [[tr.string for tr in BSRow.findAll('td')[:-1]] for BSRow in BSRows[2:]]
    return pd.DataFrame(data=rows, columns=headers)[headers[-1]]
year=10
semester=1
data=[]
i=0
while(year>0):
#    if(semester==1 and year<3):
#        semester=7
#        continue
    course=extractTable(year,semester)
    course.name=str(i)
    data.append(course)
    if(semester==1):
        semester=7
    else:
        year-=1
        semester=1
    i+=1
n=0
df_beg={}
df_end={}
index=0
print('creating df')
for i in data:
    if(index%2==0):
        if(index/2>8):
            index+=1
            continue
        df_beg[str(int(index/2))]=i
        print('num:',int(index/2),' len:',len(i))
    else:
        df_end[str(int((index+1)/2-1))]=i
    index+=1
index=0
index2=1
aux=0
while(index<len(df_beg)-1):
    while(index2<len(df_beg)):
        for i in df_beg[str(index2)].values:
            if i in df_beg[str(index)].values:
                #print(i)
                found=df_beg[str(index2)][df_beg[str(index2)]==i] #give us the subserie found
                found=found.index[0]#get index from serie
                df_beg[str(index2)]=df_beg[str(index2)].drop(found)
        index2+=1
    index+=1
    index2=index+1
df_beg=pd.DataFrame(df_beg)
df_end=pd.DataFrame(df_end)
print('ending df')
index=0
index2=2
#dict_sci={}
#while(index<len(df_beg) and index2<len(df_end)):
#print(type(df_beg[str(index)]))
three_yrs=0
four_yrs=0
five_yrs=0
while(index<len(df_beg)):
    if(index2==10):
        index+=1
        index2=index+1
    if(index2==10):
        break
    for i in df_end[str(index2)].values:
        if i in df_beg[str(index)].values:
            years=index2-index+1
            if years==3:        
                three_yrs+=1
            elif years==4:
                four_yrs+=1
            elif years==5:
                five_yrs+=1
    index2+=1
print("3 years: ",three_yrs,"\t4 years: ",four_yrs,"\t5 years:",five_yrs)

#a=set(df_beg[str(index)].values)
#b=set(df_end[str(index)].values)
#c= a&b
#print(c)
#print(df_beg[str(index)].values&df_end[str(index2)].values)
    
#if '179383' in df_beg['1'].values:
#    print('existe fuaaa')
#    df_beg=df_beg[df_beg['1']!='179383']
#print(len(df_beg['1']))

#df_end
#type(extractTable(year,semester))
#print(data)
#extractTable(year,semester).values[0]
#print(Data.html.body.table('No Sciper'))
#table = Data.html.body.table
#i=10# elementos td del html
#years=10
#sciphers_bach1_years=[]
#sciphers_bach6_years=[]
#sciphers=[]
#semester=1
#n=0#sciphersrows
#print('jjjj')

#while(years>0):
#    payloadfix['ww_x_PERIODE_ACAD'] = inputs[6].select('option')[years]['value']#decrement [10]==2007
#    payloadfix['ww_x_PERIODE_PEDAGO'] = inputs[7].select('option')[semester]['value']#[7]==bach6
#    r = requests.get(isa_dataurl, params=payloadfix)
#    Data = BeautifulSoup(r.text, 'html.parser') 
#    table = Data.html.body.table
#
#    for indX,row in enumerate(table.find_all('td')):
#        if(indX==i):
#            sciphers.insert(n,row.text)
#            i+=12
#            n+=1
#    if(semester==1):
#        sciphers_bach1_years.insert(abs(years-10),sciphers)
#        years-=1
#        semester=1
#    else: 
#        sciphers_bach6_years.insert(abs(years-10),sciphers)
#        semester=1
#    n=0
#    i=10
#    sciphers=[]
    
#print(sciphers_years[9])
#sc_bach1={'No Scipher_2007_Bach1':pd.Series(sciphers_bach1_years[0]),'No Scipher_2008_Bach1':pd.Series(sciphers_bach1_years[1]),
#   'No Scipher_2009_Bach1':pd.Series(sciphers_bach1_years[2]),'No Scipher_2010_Bach1':pd.Series(sciphers_bach1_years[3]),
#   'No Scipher_2011_Bach1':pd.Series(sciphers_bach1_years[4]),'No Scipher_2012_Bach1':pd.Series(sciphers_bach1_years[5]),
#   'No Scipher_2013_Bach1':pd.Series(sciphers_bach1_years[6]),'No Scipher_2014_Bach1':pd.Series(sciphers_bach1_years[7]),
#   'No Scipher_2015_Bach1':pd.Series(sciphers_bach1_years[8]),'No Scipher_2016_Bach1':pd.Series(sciphers_bach1_years[9])}
#print('jjjj')
#sc_bach6={'No Scipher_2007_Bach6':pd.Series(sciphers_bach6_years[0]),'No Scipher_2008_Bach6':pd.Series(sciphers_bach6_years[1]),
#   'No Scipher_2009_Bach6':pd.Series(sciphers_bach6_years[2]),'No Scipher_2010_Bach6':pd.Series(sciphers_bach6_years[3]),
#   'No Scipher_2011_Bach6':pd.Series(sciphers_bach6_years[4]),'No Scipher_2012_Bach6':pd.Series(sciphers_bach6_years[5]),
#   'No Scipher_2013_Bach6':pd.Series(sciphers_bach6_years[6]),'No Scipher_2014_Bach6':pd.Series(sciphers_bach6_years[7]),
#   'No Scipher_2015_Bach6':pd.Series(sciphers_bach6_years[8]),'No Scipher_2016_Bach6':pd.Series(sciphers_bach6_years[9])}
#print(sciphers_bach1_years[0])
#bach1=pd.DataFrame(sc_bach1)
#bach6=pd.DataFrame(sc_bach6)
#bach6